In [1]:
#Add all the imports here
import yfinance as yf
from datetime import datetime, timedelta, time
import pandas as pd
import numpy as np
from itertools import combinations
import itertools
import plotly.graph_objects as go
import time
import logging
import warnings


In [2]:
#To get data from finanace

# Silence yfinance logs
logging.getLogger("yfinance").setLevel(logging.CRITICAL)

# Silence warnings
warnings.filterwarnings("ignore")

def get_data(symbol, date,interval):
  ticker = yf.Ticker(symbol)
  start_date = datetime.strptime(date, "%Y-%m-%d")
  end_date = start_date + timedelta(days=1)
  try:
    data = ticker.history(start=start_date, end=end_date, interval=interval);
  except Exception as e:
    return None
  if data is None or data.empty:
    return None
  return data

In [3]:
#Graph helpers

def create_candlestick_figure(ticker_symbol,date):
    fig = go.Figure()

    fig.update_layout(
        title=f'{ticker_symbol} Candlestick Chart',
        xaxis_title=f"Date:{date}",
        yaxis_title="Price",
        dragmode="pan",                 # ✅ allow panning
        xaxis=dict(
            type="category",
            rangeslider=dict(visible=False),
            tickmode="array",
            showticklabels=False,  # ✅ hides x-axis labels
        )
    )

    # Enable zoom in both directions
    fig.update_xaxes(fixedrange=False)
    fig.update_yaxes(fixedrange=False)

    return fig


def update_candlestick_data(
    fig: go.Figure,
    historical_data: pd.DataFrame
):
    candle = go.Candlestick(
        x=historical_data.index,
        open=historical_data["Open"],
        high=historical_data["High"],
        low=historical_data["Low"],
        close=historical_data["Close"],
        increasing_line_color="green",
        decreasing_line_color="red",
    )
    fig.add_trace(candle)
    return fig

def add_hline(fig: go.Figure,value,annot,color="blue",annot_pos="top left"):
    fig.add_hline(
      y= value,
      line=dict(color=color, width=1, dash="dash"),
      annotation_text=f"{annot}",
      annotation_position=annot_pos
    )
    return fig

def add_vline(fig: go.Figure,date):
    start_dt = pd.Timestamp(f"{date} 10:10:00").tz_localize("Asia/Kolkata")
    end_dt   = pd.Timestamp(f"{date} 10:15:00").tz_localize("Asia/Kolkata")

    fig.add_vrect(
      x0=start_dt,
      x1=end_dt,
      fillcolor="rgba(255,0,0,0.12)",  # translucent red band
      line_width=0,
      layer="below"
    )
    return fig


def add_marker(fig: go.Figure,time,price,text):
    fig.add_annotation(
      x=time,
      y=price,
      text=text,
      showarrow=True,
      arrowhead=2,
      arrowsize=1.2,
      arrowcolor="crimson",
      ax=-40, ay=-40,
      bgcolor="rgba(255,255,255,0.8)",
      bordercolor="crimson",
      borderwidth=1,
      font=dict(size=12, color="black"),
      xref="x", yref="y"
    )
    return fig

def add_header(fig: go.Figure,text):
  fig.add_annotation(
      text=text,
      xref="paper", yref="paper",
      x=0.5, y=1.06,
      showarrow=False,
      font=dict(size=16)
  )
  return fig

def mark_entry_target_stop(fig: go.Figure,position,entry,entry_index,stop,target):
  fig = add_marker(fig, entry_index, entry, f"Entry({position}):{entry}")
  fig = add_hline(fig, stop, f"Stoploss:{stop}", "red", "top right")
  fig = add_hline(fig, target, f"Target:{target}", "green", "top right")
  return fig



In [4]:
# -------- GLOBAL COUNTERS --------
GLOBAL_STATE = {
    "win": 0,
    "loss": 0,
    "pnl" :  0,
    "tot_trade" : 0
}

In [5]:
def strategy1(ticker_symbol, date):

  global GLOBAL_STATE
  win = GLOBAL_STATE["win"]
  loss = GLOBAL_STATE["loss"]
  pnl = GLOBAL_STATE["pnl"]
  tot_trade = GLOBAL_STATE["tot_trade"]

  fig = create_candlestick_figure(ticker_symbol,date)

  # --------- LEVELS (Higher TF) ---------
  ht_data = get_data(ticker_symbol, date, "1h")
  support    = ht_data["High"].iloc[0]
  resistance = ht_data["Low"].iloc[0]

  # --------- LOWER TF DATA ---------
  data = get_data(ticker_symbol, date, "5m")
  fig = update_candlestick_data(fig, historical_data=data)

  fig = add_hline(fig,support,"support")
  fig = add_hline(fig,resistance,"resistance")
  fig = add_vline(fig,date)

  start_tf = pd.Timestamp(f"{date} 10:15:00").tz_localize("Asia/Kolkata")

  # --------- STATE ---------
  short_state = {"breakout": False, "risk": None}
  long_state  = {"breakout": False, "risk": None}

  # --------- ITERATION ---------
  i = 0
  while i < len(data):
    row = data.iloc[i]
    index = row.name

    if index < start_tf:
      i += 1
      continue

    close, high, low = row.Close, row.High, row.Low

    # =========== SHORT SETUP ==================
    if not short_state["breakout"]:
      if close > support:
        short_state["breakout"] = True
        short_state["risk"] = high
        fig = add_marker(fig, index, close, "Breakout (Short)")

    else:
      short_state["risk"] = max(short_state["risk"], high)

      if close < support:
        entry = close
        stop = short_state["risk"]
        target = entry + (entry - stop) * 2

        fig = mark_entry_target_stop(fig, "Short", entry, index, stop, target)

        #Evaluating after entry
        i += 1

        while i < len(data):
          row = data.iloc[i]
          index = row.name
          close, high, low = row.Close, row.High, row.Low

          # ---------- WIN ----------
          if high > target and target > low:
            fig = add_marker(fig, index, close, "Win")
            win += 1
            GLOBAL_STATE["win"] = win
            current_pct = round(((entry - target) / entry) * 100, 2)
            pnl += current_pct
            GLOBAL_STATE["pnl"] = pnl
            tot_trade += 1
            GLOBAL_STATE["tot_trade"] = tot_trade
            fig = add_header(fig,f"Current: Win | PnL:{current_pct}%  ||  Total: Win:{win} Loss:{loss}  PnL:{pnl}%")
            break

          # ---------- LOSS ----------
          if high > stop and stop > low:
            fig = add_marker(fig, index, close, "Loss")
            loss += 1
            GLOBAL_STATE["loss"] = loss
            current_pct = round(((entry - stop) / entry) * 100, 2)
            pnl += current_pct
            GLOBAL_STATE["pnl"] = pnl
            tot_trade += 1
            GLOBAL_STATE["tot_trade"] = tot_trade
            fig = add_header(fig,f"Current: Loss | PnL:{current_pct}%  ||  Total: Win:{win} Loss:{loss}  PnL:{pnl}%")
            break

          i += 1

        short_state = {"breakout": False, "risk": None}
        i += 1
        #continue

        break   #single trade per day

    # ================= LONG =================
    if not long_state["breakout"]:
      if close < resistance:
        long_state["breakout"] = True
        long_state["risk"] = low
        fig = add_marker(fig, index, close, "Breakout (Long)")

    else:
      long_state["risk"] = min(long_state["risk"], low)

      if close > resistance:
        entry = close
        stop = long_state["risk"]
        target = entry - (stop - entry) * 2

        fig = mark_entry_target_stop(fig, "Long", entry, index, stop, target)

        #Evaluating after entry
        i += 1
        while i < len(data):
          row = data.iloc[i]
          index = row.name
          close, high, low = row.Close, row.High, row.Low

          # ---------- WIN ----------
          if high > target and target > low:
            fig = add_marker(fig, index, close, "Win")
            win += 1
            GLOBAL_STATE["win"] = win
            current_pct = round(((target - entry) / entry) * 100, 2)
            pnl += current_pct
            GLOBAL_STATE["pnl"] = pnl
            tot_trade += 1
            GLOBAL_STATE["tot_trade"] = tot_trade
            fig = add_header(fig,f"Current: Win | PnL:{current_pct}%  ||  Total: Win:{win} Loss:{loss}  PnL:{pnl}%")
            break

          # ---------- LOSS ----------
          if high > stop and stop > low:
            fig = add_marker(fig, index, close, "Loss")
            loss += 1
            GLOBAL_STATE["loss"] = loss
            current_pct = round(((stop - entry) / entry) * 100, 2)
            pnl += current_pct
            GLOBAL_STATE["pnl"] = pnl
            tot_trade += 1
            GLOBAL_STATE["tot_trade"] = tot_trade
            fig = add_header(fig,f"Current: Loss | PnL:{current_pct}%  ||  Total: Win:{win} Loss:{loss}  PnL:{pnl}%")
            break

          i += 1

        long_state = {"breakout": False, "risk": None}
        i += 1
        #continue

        break   #single trade per day

    i += 1

  return fig


In [6]:
tz="Asia/Kolkata"

def run_range(ticker_symbol, days_back, show_fig=False):
    today = pd.Timestamp.now(tz=tz).normalize()
    start = today - pd.Timedelta(days=days_back-1)
    dates = pd.date_range(start=start, end=today, freq="D", tz=tz)

    total = len(dates)
    print(f"Running {ticker_symbol} from {dates[0].date()} to {dates[-1].date()} ({total} days)…")

    auto = False
    for i, dt in enumerate(dates, start=1):
        date_str = dt.strftime("%Y-%m-%d")  # matches your strategy1 signature
        if show_fig:
          print(f"\n[{i}/{total}] strategy1({ticker_symbol!r}, {date_str})")

        try:
            fig = strategy1(ticker_symbol, date_str)  # <-- your existing function
            if show_fig:
              fig.show(config={"scrollZoom": True})
        except Exception as e:
            pass

    print(f"Total: Win:{GLOBAL_STATE["win"]} Loss:{GLOBAL_STATE["loss"]}  PnL:{GLOBAL_STATE["pnl"]}%\n")

In [7]:
def get_ticker_list():
    # Authenticate (Colab)
    from google.colab import auth
    auth.authenticate_user()

    import gspread
    import pandas as pd
    from google.auth import default

    # Authorize
    creds, _ = default()
    gc = gspread.authorize(creds)


    # Open sheet
    sheet = gc.open_by_url(
        "https://docs.google.com/spreadsheets/d/1SAS2_nj_BN22vsYMbpG79MgbiBLI1YaVkRejYMWe3hE/edit"
    ).sheet1

    # Read data into DataFrame
    df = pd.DataFrame(sheet.get_all_records())

    # Flatten → clean → append ".NS"
    tickers = (
        df
        .astype(str)                 # ensure strings
        .stack()                     # flatten column-wise
        .str.strip()                 # remove whitespace
        .loc[lambda s: s != ""]       # remove empty strings
        .apply(lambda s: s if s.endswith(".NS") else s + ".NS")
        .tolist()
    )

    return tickers


In [8]:
def run_multi_range(ticker_symbol, days_back, show_fig=False):
  for i in ticker_symbol:
    run_range(i, days_back, show_fig)

In [9]:
#Main entry

run_multi_range(get_ticker_list(),60, False)


# For testing single over range
#ticker_symbol = "TITAN.NS"
#run_range(ticker_symbol,60,True)

# For testing single
#ticker_symbol = "HDFCBANK.NS"
#date = "2025-12-19"
#fig = strategy1(ticker_symbol, date);
#fig.show(config={"scrollZoom": True})


Running TITAN.NS from 2025-10-29 to 2025-12-27 (60 days)…
Total: Win:5 Loss:13  PnL:-1.6900000000000002%

Running IDFCFIRSTB.NS from 2025-10-29 to 2025-12-27 (60 days)…
Total: Win:9 Loss:32  PnL:-4.0200000000000005%

Running PFC.NS from 2025-10-29 to 2025-12-27 (60 days)…
Total: Win:16 Loss:44  PnL:-3.3400000000000007%

Running MARICO.NS from 2025-10-29 to 2025-12-27 (60 days)…
Total: Win:23 Loss:60  PnL:-4.239999999999998%

Running IRFC.NS from 2025-10-29 to 2025-12-27 (60 days)…
Total: Win:27 Loss:76  PnL:-5.930000000000001%

Running HINDALCO.NS from 2025-10-29 to 2025-12-27 (60 days)…
Total: Win:36 Loss:85  PnL:-5.340000000000001%

Running AXISBANK.NS from 2025-10-29 to 2025-12-27 (60 days)…
Total: Win:41 Loss:102  PnL:-7.91%

Running RECLTD.NS from 2025-10-29 to 2025-12-27 (60 days)…
Total: Win:45 Loss:120  PnL:-10.42%

Running BHEL.NS from 2025-10-29 to 2025-12-27 (60 days)…
Total: Win:50 Loss:130  PnL:-10.38%

Running HINDZINC.NS from 2025-10-29 to 2025-12-27 (60 days)…
Total: Wi

KeyboardInterrupt: 